In [4]:
# Imports

import torch

In [5]:
# Let's start download the training dataset

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

/bin/bash: wget: command not found


In [6]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [7]:
print('Number of characters in the datasets: ', len(text))

Number of characters in the datasets:  1115394


In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encode the text to indices
decode = lambda l: ''.join([itos[i] for i in l]) # decode the index list to text

print(encode("hi there"))
print(decode([46, 47, 1, 58, 46, 43, 56, 43]))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [10]:
# Convert text to numerical type. We are using character level tokenizer for this project

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [11]:
# Separate data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

len(train_data), len(val_data)

(1003854, 111540)

In [12]:
# Let's define block size: 
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [14]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will we process in parallel?
block_size = 8 # What is the maximum context length for prediction?

def get_batch(split):
    # generate small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix]) # stack function puts the one dimensional tensor into a multi-dimensional tensor
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print()
print('targets: ')
print(yb.shape)
print(yb)
print('---------------------------------------------')
print()

for b in range(batch_size): # Batch dimension
    for t in range(block_size): # Time dimension
        context = xb[b][:t+1] # -> equals to: xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------------------------------------------

when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56

# Bigram model

In [15]:
# Let's implement BiGram language model first:

torch.manual_seed(1337)

import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Embedding layer for token embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        """
        Forward pass of the language model.
        
        Args:
            idx: Tensor of shape (B, T) containing token indices.
            targets: Tensor of shape (B, T) containing target token indices, or None.
        
        Returns:
            logits: Tensor of shape (B, T, C) containing the raw, unnormalized scores for each token.
            loss: Computed loss value if targets are provided, else None.
        """
        # Compute logits from token embeddings
        logits = self.token_embedding_table(idx) # (B, T, C)

        # Compute loss if targets are provided
        if targets is None:
            loss = None
        else:
            ''' https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#crossentropyloss:
                    The description of the crossEntropy function states the it needs its inputs in a way that the,
                    'C' - channels are in the 2nd dimension, so that is why we need to reshape our tensors
            '''
            '''
                logits tensor has shape (B, T, C) after the forward pass, 
                where B is the batch size, 
                T is the sequence length, 
                and C is the number of classes (vocab size)
            '''
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T) # We could use -1 as well
            loss = F.cross_entropy(logits, targets)
    
        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        Generate new tokens from the model.
        
        Args:
            idx: Tensor of shape (B, T) containing the current context indices.
            max_new_tokens: Number of new tokens to generate.
        
        Returns:
            idx: Tensor of shape (B, T + max_new_tokens) containing the extended sequence.
        """
        for _ in range(max_new_tokens):
            # Get the predictions
            logits, _ = self.forward(idx)
            # Focus only on the last time step
            logits = logits[:, -1, :] # (B, C)
            # Apply softmax to get the probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist())) # -> equals: decode(torch.tensor(m.generate(idx, max_new_tokens=100)).view(-1).tolist())

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [16]:
# Create a PyTorch optimizer for train the model: (We used to use SGD optimizer, but Adam is more efficient)
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [17]:
# Let's train the model
batch_size = 32

for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # zero out the gradients
    loss.backward() # calculate the new gradients
    optimizer.step() # update the gradients

    if steps % 1000 == 0:
        print(loss.item())

4.692410945892334
3.7637593746185303
3.2342257499694824


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


### The mathematical trick in self attention:

In [ ]:
# Consider the following example:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b, t] = mean_{i<=t} -> x[b, i]
xbow = torch.zeros((B,T,C)) # bow - bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

x[0], xbow[0]

# This way, it is really inefficient!!

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [ ]:
# Bottom-triangle matrix
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
# Using the tril function to achive the moving window of using only the first item, then the first and the second, etc...

torch.manual_seed(1337)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[5., 7.],
        [2., 0.],
        [5., 3.]])
--
c=
tensor([[ 5.,  7.],
        [ 7.,  7.],
        [12., 10.]])


In [ ]:
# To average on all of this in efficient way:
# For example, the third row will have (0.3333, 0.3333, 0.3333), so by multiplying with this, it will return the average of those 3 numbers

torch.manual_seed(1337)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[5., 7.],
        [2., 0.],
        [5., 3.]])
--
c=
tensor([[5.0000, 7.0000],
        [3.5000, 3.5000],
        [4.0000, 3.3333]])


In [ ]:
# Let's go back to the initial averaging:

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
print(wei)

xbow2 = wei @ x # (T, T) @ (B, T, C) -(broadcasting)-> (B, T, T) @ (B, T, C) ----> (B, T, C)
xbow2[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [ ]:
# version 3: use SoftMax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # all the elements in the bottom-triangle matrix where it is 0, it will be -inf
wei = F.softmax(wei, dim=-1) # apply softmax on the elements / When you use dim=-1, it means that the softmax operation will be applied along the last dimension of the tensor.
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [ ]:
# version 4.: Self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16

key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # B, T, 16
q = query(x) # B, T, 16
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) 
wei = F.softmax(wei, dim=-1) 

v = value(x)
out = wei @ v

out.shape, wei.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 8]))

#### Notes:
- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. If it is not optimized, it will take huge numbers and after a while it will converge to one-hot vectors. If the numbers are higher, the softmax will sharpen and converge to the highest number of the tensor, means the softmax gets to picky. Illustration below

In [ ]:
k = torch.randn(B,T,head_size) # unit gaussian inputs, because of the randn
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5 # -> optimization from the paper: 'All you need is attention'

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var() # without the '* head_size**-0.5' optimization, the 'wei's' var will be around the head_size, but with the optimization the k and q's var will be preserved in wei

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

### Let's implement LayerNorm:
- https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html

Here is the BatchNorm from Makemore:

In [ ]:
class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(0, keepdim=True) # batch mean
    xvar = x.var(0, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
      return [self.gamme, self.beta]

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100) # batc size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

The mean and the variance will apply to gaussian distribution with around 0 mean and 1 variance:

In [ ]:
x[:, 0].mean(), x[:, 0].std() # mean, std of one feature across all batch inputs

(tensor(1.4901e-08), tensor(1.0000))

In [ ]:
class LayerNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # layer mean, CHANGED the axis to 1
    xvar = x.var(1, keepdim=True) # batch variance, CHANGED the axis to 1
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
      return [self.gamme, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batc size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

The mean and the variance will apply to gaussian distribution with around 0 mean and 1 variance for layers, after we changed the xmean's and xvar's axis from batch to layer:

In [ ]:
x[0, :].mean(), x[0, :].std() # mean, std of one feature across all batch inputs

(tensor(-3.5763e-09), tensor(1.0000))